In [ ]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
from typing import List, Dict

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

# processed 10min average measurement data
df_1m = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_cal_corr_acropolis.parquet"))

df_1h = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "1h_cal_corr_acropolis.parquet"))


In [10]:


test = {"2":[3,4],
        "3":[4,5]}

def test1(test:dict):
    print(test)
    
test1(test)

{'2': [3, 4], '3': [4, 5]}


In [ ]:
# Utility

# dates = {
    # "system_id": [start_date, end_date],
    # "system_id": [start_date, end_date]
# }

def extract_site_data(dates:dict):
    pass

In [ ]:
# Maisach

start_date = datetime(2024, 6, 8, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 6, 10, 23, 59, 59).replace(tzinfo=timezone.utc)


df_1m = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_cal_corr_acropolis.parquet")).filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()
